# EXploration of features

In [1]:
import os
import glob
import nibabel as nib
import pandas as pd
from data_loader_folders import load_patient_files

In [3]:
def check_qsm_dimensions(root_dir, folder_type, output_csv_path):
    """
    Iterates over the patient folders in root_dir (using load_patient_files),
    loads the QSM image for each patient, and checks if any of its voxel spacings
    (x, y, z) are different than 1.
    
    For each dimension that is not equal to 1, a record is added with:
      - PatientID
      - FolderType (e.g., "baseline" or "follow_up")
      - Year (folder name)
      - Dimension (e.g., 'x', 'y', or 'z')
      - Spacing value
    
    The collected records are then saved to a CSV file at output_csv_path.
    
    Parameters
    ----------
    root_dir : str
        Path to the root folder (e.g., baseline or follow-up).
    folder_type : str
        Label indicating the folder type ("baseline" or "follow_up").
    output_csv_path : str
        File path for the output CSV.
    """
    # Call your existing function to get the patient records.
    records = load_patient_files(root_dir, folder_type)
    
    results = []
    # Define dimension labels for clarity.
    dim_labels = ['x', 'y', 'z']
    
    for rec in records:
        qsm_path = rec["QSM"]
        # Load the QSM image using nibabel.
        img = nib.load(qsm_path)
        # Get the voxel spacing for the first 3 dimensions.
        spacing = img.header.get_zooms()[:3]
        
        # Check each spatial dimension.
        for label, sp in zip(dim_labels, spacing):
            if sp != 1:
                results.append({
                    "PatientID": rec["PatientID"],
                    "FolderType": rec["FolderType"],
                    "Year": rec["Year"],
                    "Dimension": label,
                    "Spacing": sp
                })
    
    # Create a DataFrame from the results and write to CSV.
    df = pd.DataFrame(results)
    df.to_csv(output_csv_path, index=False)


In [4]:
baseline_dir = "/home/jbetancur/Desktop/codes/python_qsm/exploratory_pipeline/data_automatization/clean_baseline"
followup_dir = "/home/jbetancur/Desktop/codes/python_qsm/exploratory_pipeline/data_automatization/clean_follow_up"

check_qsm_dimensions(baseline_dir, "baseline", output_csv_path="/home/jbetancur/Desktop/codes/clustering/feature_extraction/output/baseline_spacing.csv")
check_qsm_dimensions(followup_dir, "followup", output_csv_path="/home/jbetancur/Desktop/codes/clustering/feature_extraction/output/follow_up_spacing.csv")